In [2]:
#Firsty we import the packages we may need for the data wrangling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

In [4]:
#First we import all the csv files we need for the work and make a quick look at them
df_features = pd.read_csv('features.csv')
df_sample = pd.read_csv('sampleSubmission.csv')
df_stores = pd.read_csv('stores.csv')
df_train = pd.read_csv('train.csv')
df_stock_price = pd.read_csv('WMT.csv')

During this work, we are trying to predict the weekly sales Walmart could have by using the dataset we were able to see in Kaggle. We have 5 different datasets. The widest one is the feature dataset that contains the store, with the temperature, fuel price, markdown (promotions going on), CPI, unemployment and if the date was Holiday or not.

We have another dataset which contains the type and the size for each store.

Another dataset with name Train contains as additional information the weekly sales that is the varible we are going to try to predict (this would be our dependant variable) and all other variables would be independant variables.

Finally, we decided to include Walmart's Stock price.

To give a summarize, on this data wrangling we have the idea to organize the data. We are going to check we have the desirible types on the dataframe, if we need this, change to categorical variables, so the machine learning process would be easier, merge all the tables in one, and handle the missing values, so when we arrive to the modelling part, the process would be easier.

# Data Overview

On the Data Overview, we are going to take a look to each data set. We are going to see how the dataset looks like, the types of the column (and change the type if we need to do so), and also have an understanding of the amount of missing values per each column

### Features

In [3]:
#Overview of the features dataset
df_features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [4]:
# Have an understading of how many NaN values are per column with the type of each column with info method
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
Store           8190 non-null int64
Date            8190 non-null object
Temperature     8190 non-null float64
Fuel_Price      8190 non-null float64
MarkDown1       4032 non-null float64
MarkDown2       2921 non-null float64
MarkDown3       3613 non-null float64
MarkDown4       3464 non-null float64
MarkDown5       4050 non-null float64
CPI             7605 non-null float64
Unemployment    7605 non-null float64
IsHoliday       8190 non-null bool
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


## Stores

In [5]:
#Overview of the Stores Dataset
df_stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [6]:
# Have an understading of how many NaN values are per column with the type of each column with info method
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
Store    45 non-null int64
Type     45 non-null object
Size     45 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


## Train

In [7]:
#Overview of the Train dataset
df_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [8]:
# Have an understading of how many NaN values are per column with the type of each column with info method
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
Store           421570 non-null int64
Dept            421570 non-null int64
Date            421570 non-null object
Weekly_Sales    421570 non-null float64
IsHoliday       421570 non-null bool
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


## Stock Price

In [9]:
#Overview of the stock price 
df_stock_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-02-05,52.770000,53.529999,52.759998,53.450001,41.123001,15545800
1,2010-02-08,53.380001,53.540001,52.919998,52.930000,40.722912,10505900
2,2010-02-09,53.209999,53.619999,52.820000,53.250000,40.969128,10825000
3,2010-02-10,53.270000,53.500000,52.959999,53.240002,40.961433,9408400
4,2010-02-11,53.130001,53.450001,52.910000,53.080002,40.838329,11579600


In [10]:
#Extract the columns I am interested. In this case, Date and Close price
df_stock_price = df_stock_price[['Date', 'Close']]

In [11]:
# Overview of the dataset I am going to use
df_stock_price.head()

,Date,Close
0,2010-02-05,53.450001
1,2010-02-08,52.930000
2,2010-02-09,53.250000
3,2010-02-10,53.240002
4,2010-02-11,53.080002


In [12]:
#See the types
df_stock_price.dtypes

Date      object
Close    float64
dtype: object

# Convert all the date's columns to datetime for future use

During this stage, I will firstly convert all the dates columns into datetime in order to do the merge among all the columns.

In [13]:
df_features['Date'] =  pd.to_datetime(df_features['Date'])
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_stock_price['Date'] = pd.to_datetime(df_stock_price['Date'])

# Start the merging of all the data frames to have all the information in one big data frame

#### By looking the data, we are taking off the test data frame since there is no concordance with the dates of the other data frames. 

In [14]:
'''We start merging the features data frame with the train dataframe. Left Join to include values, and on the date, 
store and Is Holiday column as the concordance column between the two data frames'''
df_features_train = pd.merge(df_train, df_features,how = 'inner', on = ['Date', 'Store', 'IsHoliday'])

In [15]:
df_features_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,2,2010-02-05,50605.27,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
2,1,3,2010-02-05,13740.12,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
3,1,4,2010-02-05,39954.04,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
4,1,5,2010-02-05,32229.38,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106


In [16]:
df_features_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,2,2010-02-05,50605.27,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
2,1,3,2010-02-05,13740.12,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
3,1,4,2010-02-05,39954.04,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
4,1,5,2010-02-05,32229.38,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106


In [17]:
'''We continued merging the df_walmart data frame with the stock price dataframe. Left Join to include values and the
date column  as the concordance column between the two data frames'''
df_features_train_stock = pd.merge(df_features_train, df_stock_price,how = 'inner', on = ['Date'])

In [18]:
#See the new dataframe 
df_features_train_stock.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Close
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001
1,1,2,2010-02-05,50605.27,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001
2,1,3,2010-02-05,13740.12,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001
3,1,4,2010-02-05,39954.04,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001
4,1,5,2010-02-05,32229.38,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001


In [19]:
# It is time now to merge the last data frame with the store column
df = pd.merge(df_features_train_stock, df_stores, how = 'inner', on = ['Store'])

In [20]:
#Have an overview of the new data frame
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Close,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001,A,151315
1,1,2,2010-02-05,50605.27,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001,A,151315
2,1,3,2010-02-05,13740.12,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001,A,151315
3,1,4,2010-02-05,39954.04,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001,A,151315
4,1,5,2010-02-05,32229.38,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,53.450001,A,151315


In [21]:
#See the data frame types to confirm all is in order
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409727 entries, 0 to 409726
Data columns (total 17 columns):
Store           409727 non-null int64
Dept            409727 non-null int64
Date            409727 non-null datetime64[ns]
Weekly_Sales    409727 non-null float64
IsHoliday       409727 non-null bool
Temperature     409727 non-null float64
Fuel_Price      409727 non-null float64
MarkDown1       147698 non-null float64
MarkDown2       111033 non-null float64
MarkDown3       134202 non-null float64
MarkDown4       132337 non-null float64
MarkDown5       148449 non-null float64
CPI             409727 non-null float64
Unemployment    409727 non-null float64
Close           409727 non-null float64
Type            409727 non-null object
Size            409727 non-null int64
dtypes: bool(1), datetime64[ns](1), float64(11), int64(3), object(1)
memory usage: 53.5+ MB


# Handling Missing Values

On this part, I will handle the missing values. From the info method, we can see there are many missing values we need to either remove or fill.

In [22]:
#Drop columns with many NaN Values
#I will use a threshold of 30%. If there is 30% of missing values or more, I would be removing those columns
threshold = 0.3
df = df[df.columns[df.isnull().mean() < threshold]]

In [23]:
#Drop rows with many NaN Values
df = df.loc[df.isnull().mean(axis = 1) < threshold]

In [24]:
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Close,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,211.096358,8.106,53.450001,A,151315
1,1,2,2010-02-05,50605.27,False,42.31,2.572,211.096358,8.106,53.450001,A,151315
2,1,3,2010-02-05,13740.12,False,42.31,2.572,211.096358,8.106,53.450001,A,151315
3,1,4,2010-02-05,39954.04,False,42.31,2.572,211.096358,8.106,53.450001,A,151315
4,1,5,2010-02-05,32229.38,False,42.31,2.572,211.096358,8.106,53.450001,A,151315


# Fill of the NaN Values

In [25]:
'''Replace the NaN Values
Close price: It should have been a weekend day where the stock is not traded. Replace NaN values with the value
of the days before
Unemployment: Replace this value with the mean
Weekly Sales: Drop those rows. It is a bad idea to fill the variable we are trying to predict
CPI: Fill NaN values with the mean
Weekly_Sales = Fill NaN values with the mean
'''

'Replace the NaN Values\nClose price: It should have been a weekend day where the stock is not traded. Replace NaN values with the value\nof the days before\nUnemployment: Replace this value with the mean\nWeekly Sales: Drop those rows. It is a bad idea to fill the variable we are trying to predict\nCPI: Fill NaN values with the mean\nWeekly_Sales = Fill NaN values with the mean\n'

In [26]:
#Replace Close price using backfill methid
df['Close'].fillna(method = 'bfill', inplace = True)

In [27]:
#Replace Unemployment with the mean
df['Unemployment'].fillna(df['Unemployment'].mean(), inplace = True)

In [28]:
# Fill CPA values with the mean
df['CPI'].fillna(df['CPI'].mean(), inplace = True)

In [29]:
df = df.dropna()

In [30]:
df.to_csv('walmart_sales.csv')

## Change the department to a categorical variable

In [31]:
#Aggregate the department, store and type as a dummy variable to get it as a possible exploratory variable
df_dummies = pd.get_dummies(df, columns = ['Dept', 'Store', 'Type'])

In [32]:
#Overview of our organized dataframe
df_dummies.head()

,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Close,Size,Dept_1,...,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45,Type_A,Type_B,Type_C
0,2010-02-05,24924.50,False,42.31,2.572,211.096358,8.106,53.450001,151315,1,...,0,0,0,0,0,0,0,1,0,0
1,2010-02-05,50605.27,False,42.31,2.572,211.096358,8.106,53.450001,151315,0,...,0,0,0,0,0,0,0,1,0,0
2,2010-02-05,13740.12,False,42.31,2.572,211.096358,8.106,53.450001,151315,0,...,0,0,0,0,0,0,0,1,0,0
3,2010-02-05,39954.04,False,42.31,2.572,211.096358,8.106,53.450001,151315,0,...,0,0,0,0,0,0,0,1,0,0
4,2010-02-05,32229.38,False,42.31,2.572,211.096358,8.106,53.450001,151315,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
df_dummies.to_csv('walmart_sales_dummies.csv')

In [ ]:
pd.set_option('max_rows', None )
df_dummies